In [1]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.4 MB/s eta 0:00:00


###Conjunto de datos de rendiciones internas

In [4]:
import pandas as pd
import numpy as np
import random
from faker import Faker  # <--- IMPORTACIÓN FALTANTE

# 1. Configuración de Faker y Semillas
fake = Faker('es_AR') # Usamos 'fake' para que coincida con el resto del script
num_rendiciones = 30
seed = 42
random.seed(seed)
np.random.seed(seed)

# 2. Generación de montos
# Nota: Cambié num_contratos por num_rendiciones para evitar errores
montos = np.random.uniform(1000, 50000, num_rendiciones)
montos_formateados = [np.round(monto, 2) for monto in montos]

data_rendiciones = {
    'ID_Rendicion': range(1, num_rendiciones + 1),
    'Fecha_Presentacion': pd.to_datetime([fake.date_between(start_date='-365d', end_date='today') for _ in range(num_rendiciones)]),
    'Empleado': [fake.name() for _ in range(num_rendiciones)],
    'Departamento': [random.choice(['Ventas', 'Marketing', 'Compras', 'RRHH', 'Finanzas']) for _ in range(num_rendiciones)],
    'Tipo_Gasto': [random.choice(['Viáticos', 'Material de Oficina', 'Almuerzo', 'Transporte', 'Otros']) for _ in range(num_rendiciones)],
    'Descripcion': [random.choice(['gastos', 'viaticos', 'compras', 'gastos_generales']) for _ in range(num_rendiciones)],
    'Monto': montos_formateados,
    'Justificante_Adjunto': [random.choice(['Sí', 'No', 'Parcial']) for _ in range(num_rendiciones)],
    'Estado_Aprobacion': [random.choice(['Aprobado', 'Pendiente', 'Rechazado']) for _ in range(num_rendiciones)],
    'Es_Sospechoso': np.zeros(num_rendiciones, dtype=int)
}

# 3. Creación del DataFrame y Reglas de Sospecha
df_rendiciones1 = pd.DataFrame(data_rendiciones)

# Aplicar lógica de auditoría
df_rendiciones1['Es_Sospechoso'] = np.where(
    (df_rendiciones1['Justificante_Adjunto'] == 'No') | (df_rendiciones1['Estado_Aprobacion'] == 'Rechazado'),
    1,
    0
)

# 4. Guardado Local
nombre_archivo_csv = 'df_rendiciones1.csv'
df_rendiciones1.to_csv(nombre_archivo_csv, index=False)

print(f"✅ El DataFrame se ha guardado exitosamente en '{nombre_archivo_csv}'")
df_rendiciones1.head()

✅ El DataFrame se ha guardado exitosamente en 'df_rendiciones1.csv'


,ID_Rendicion,Fecha_Presentacion,Empleado,Departamento,Tipo_Gasto,Descripcion,Monto,Justificante_Adjunto,Estado_Aprobacion,Es_Sospechoso
0,1,2025-07-19,Joaquina Mora Lopez,Ventas,Almuerzo,viaticos,19352.47,Parcial,Pendiente,0
1,2,2025-05-25,Ciro Benitez Fernandez,Ventas,Almuerzo,gastos,47585.00,No,Rechazado,1
2,3,2025-05-31,Luca Ciro Muñoz,Compras,Material de Oficina,gastos_generales,36867.70,No,Aprobado,1
3,4,2025-09-28,Juana Lorenzo Rodriguez Gomez,Marketing,Material de Oficina,compras,30334.27,Parcial,Pendiente,0
4,5,2025-03-19,Maria Belen Garcia,Marketing,Almuerzo,gastos_generales,8644.91,No,Pendiente,1


### Algoritmo para fraude en rendicion de cuentas

In [8]:
import pandas as pd
import numpy as np
import os
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# 1. Carga de Datos
archivo_csv = 'df_rendiciones1.csv'
if os.path.exists(archivo_csv):
    df_rendiciones = pd.read_csv(archivo_csv)
    df_rendiciones["Fecha_Presentacion"] = pd.to_datetime(df_rendiciones["Fecha_Presentacion"])
else:
    print(f"Error: No se encontró {archivo_csv}")
    raise FileNotFoundError

# 2. Re-inyección de lógica para asegurar la salida (Simulación)
# Para que la salida coincida con tu ejemplo de 9 registros en el test set
df_rendiciones['Es_Sospechoso'] = df_rendiciones['Es_Sospechoso'].fillna(0)

# 3. Ingeniería de Características (Features)
df_rendiciones['Monto_Promedio_Tipo'] = df_rendiciones.groupby('Tipo_Gasto')['Monto'].transform('mean')
df_rendiciones['Monto_Alto_Relativo'] = np.where(df_rendiciones['Monto'] > df_rendiciones['Monto_Promedio_Tipo'] * 2, 1, 0)
df_rendiciones['Sin_Justificante_Alto_Monto'] = np.where((df_rendiciones['Justificante_Adjunto'] == 'No') & (df_rendiciones['Monto'] > 200), 1, 0)
df_rendiciones['Otros_Monto_Elevado'] = np.where((df_rendiciones['Tipo_Gasto'] == 'Otros') & (df_rendiciones['Monto'] > 500), 1, 0)
df_rendiciones['Dia_Semana'] = df_rendiciones['Fecha_Presentacion'].dt.dayofweek
df_rendiciones['Viaticos_Fin_Semana'] = np.where((df_rendiciones['Tipo_Gasto'] == 'Viáticos') & (df_rendiciones['Dia_Semana'] >= 4), 1, 0)
df_rendiciones['Gastos_Frecuentes'] = 0 # Inicializado según tu salida
df_rendiciones['Descripcion_Generica_Monto_Alto'] = np.where(df_rendiciones['Monto'] > 300, 1, 0)

# 4. Preparación de variables
features = ['Monto', 'Monto_Alto_Relativo', 'Sin_Justificante_Alto_Monto',
            'Otros_Monto_Elevado', 'Gastos_Frecuentes', 'Viaticos_Fin_Semana',
            'Descripcion_Generica_Monto_Alto', 'Monto_Promedio_Tipo']

X = df_rendiciones[features].fillna(0)
y = df_rendiciones['Es_Sospechoso']

# --- INICIO DE SALIDA REQUERIDA ---
print("Código para Detección de Fraude Interno en Rendiciones de Cuenta")
print("\nEjemplo de un Sistema de Detección de Fraude Interno (Modelo de IA):")

# 5. Validación Cruzada
modelo = RandomForestClassifier(random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(modelo, X, y, cv=cv)
print(f"\nPrecisión de Validación Cruzada: {np.mean(scores):.4f} (+/- {np.std(scores):.4f})")

# 6. Entrenamiento y Evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

print("\nEvaluación en el conjunto de prueba:")
print(f"Precisión: {accuracy_score(y_test, y_pred)}")
print("\nReporte de Clasificación:\n", classification_report(y_test, y_pred, target_names=['No Sospechoso', 'Sospechoso']))
print("\nMatriz de Confusión:\n", confusion_matrix(y_test, y_pred))

# Repetición del bloque según tu formato
print("\nCódigo para Detección de Fraude Interno en Rendiciones de Cuenta")
print("\nEjemplo de un Sistema de Detección de Fraude Interno (Modelo de IA):")
print("\nEvaluación del Modelo de Detección de Fraude Interno:")
print(f"Precisión: {accuracy_score(y_test, y_pred)}")
print("\nReporte de Clasificación:\n", classification_report(y_test, y_pred, target_names=['No Sospechoso', 'Sospechoso']))
print("\nMatriz de Confusión:\n", confusion_matrix(y_test, y_pred))

# 7. Identificación de Sospechosos
print("\nEjemplo de cómo el Sistema podría identificar rendiciones sospechosas:")
print("\nRendiciones de Cuenta Marcadas como Sospechosas por el Sistema:")

df_test = df_rendiciones.loc[X_test.index].copy()
df_test['Prediccion_Sospechoso'] = y_pred
rendiciones_sospechosas = df_test[df_test['Prediccion_Sospechoso'] == 1][
    ['ID_Rendicion', 'Fecha_Presentacion', 'Empleado', 'Departamento', 'Tipo_Gasto', 'Monto',
     'Justificante_Adjunto', 'Estado_Aprobacion', 'Es_Sospechoso', 'Prediccion_Sospechoso']]

print(rendiciones_sospechosas)
print("\nEstas rendiciones podrían requerir una revisión más exhaustiva.")

# 8. Importancia de Características
if hasattr(modelo, 'feature_importances_'):
    importancia = pd.DataFrame({'Caracteristica': features, 'Importancia': modelo.feature_importances_})
    importancia = importancia.sort_values(by='Importancia', ascending=False)
    print("\nAnálisis de Importancia de las Características:")
    print(importancia)

Código para Detección de Fraude Interno en Rendiciones de Cuenta

Ejemplo de un Sistema de Detección de Fraude Interno (Modelo de IA):

Precisión de Validación Cruzada: 0.7000 (+/- 0.1944)

Evaluación en el conjunto de prueba:
Precisión: 0.7777777777777778

Reporte de Clasificación:
                precision    recall  f1-score   support

No Sospechoso       0.67      1.00      0.80         4
   Sospechoso       1.00      0.60      0.75         5

     accuracy                           0.78         9
    macro avg       0.83      0.80      0.78         9
 weighted avg       0.85      0.78      0.77         9


Matriz de Confusión:
 [[4 0]
 [2 3]]

Código para Detección de Fraude Interno en Rendiciones de Cuenta

Ejemplo de un Sistema de Detección de Fraude Interno (Modelo de IA):

Evaluación del Modelo de Detección de Fraude Interno:
Precisión: 0.7777777777777778

Reporte de Clasificación:
                precision    recall  f1-score   support

No Sospechoso       0.67      1.00     